In [1]:
import numpy as np
import scipy.linalg as lina

import scipy.stats as stat

import QuantumToolbox.operators as qOps
import QuantumToolbox.liouvillian as liou
import QuantumToolbox.states as states

import RMT_statistics.Modules.Distributions as RMTdist

import matplotlib.pyplot as plt

import Plotting.Functions as pltFncs
import Plotting.SimplePlots as sPlt

# Eigenvector Statistics of Kicked-top of Different Symmetries
---
    M Kus, J Mostowski and F Haake
    Journal of Physics A: Mathematical and General, Volume 21, Number 22
    https://iopscience.iop.org/article/10.1088/0305-4470/21/22/006
---

 ## 1.COE

Parameters, Hamiltonian, and the Unitaries

In [ ]:
# Parameters
p = 1.7
j = 100
la = 6

# required operators
jz = qOps.Jz(j)
jy = qOps.Jy(j)
jzs = jz@jz

# Hamiltonians
h_o = p*jy
v_o = (la/(2*j + 1))*(jzs)

# Unitaries
U_ho = liou.Liouvillian(h_o)
U_o = liou.Liouvillian(v_o)@U_ho

# Eigenvectors of the U_o
vecs = lina.eig(U_o.A)[1]

# components of eigenvectors in the Jz basis
components = (np.abs(vecs.flatten()))**2

In [ ]:
# Plotting
bins = np.logspace(-5, -1, num=50, base=10.0)

# quasi-continuous PDF and linear axes
sPlt.pltEig(components, Dims=((2*j) + 1), Bins=bins, Step=False, logScale=False)

In [ ]:
x_o = []
x_s = []
x_u = []

jlist = np.arange(0.5,8.5,0.5)

bins = np.logspace(-5, -1, num=50, base=10.0)
dig = (np.histogram(components, bins, density=True)[0])

bin_means = (np.histogram(bins, bins, weights=bins)[0] / np.histogram(bins, bins)[0])

COE = [RMTdist.EigenVectorDist(X, int(2 * j) + 1, 1) for X in bin_means]
CUE = [RMTdist.EigenVectorDist(X, int(2 * j) + 1, 2) for X in bin_means]
CSE = [RMTdist.EigenVectorDist(X, int(2 * j) + 1, 4) for X in bin_means]

xo = stat.chisquare(dig,f_exp=COE)
xu = stat.chisquare(dig,f_exp=CUE)
xs = stat.chisquare(dig,f_exp=CSE)
    
print(xu)
print(xo)

In [ ]:
# discrete PDF and log axes
sPlt.pltEig(components, Dims=((2*j) + 1), Bins=bins, Step=True, logScale=True)

 ## 2.CUE

Parameters, Hamiltonian, and the Unitaries

In [ ]:
# Parameters
pu = 1.7
ju = 100
ku = 6
kpu = 0.5

# required operators
jzu = qOps.Jz(ju)
jxu = qOps.Jx(ju)
jyu = qOps.Jy(ju)
jzsu = jzu@jzu
jxsu = jxu@jxu

# Hamiltonians
h_u = pu*jy
v_u1 = (ku/(2*j))*(jzsu)
v_u2 = (kpu/(2*j))*(jxsu)

# Unitaries
U_hu = liou.Liouvillian(h_u)
U_u1 = liou.Liouvillian(v_u1)@U_hu
U_u = liou.Liouvillian(v_u2)@U_u1

# Eigenvectors of the U_u
vecsu = lina.eig(U_u.toarray())[1]

# components of eigenvectors in the Jz basis
components = (np.abs(vecsu.flatten()))**2

In [ ]:
# discrete PDF and log axes
sPlt.pltEig(components, Dims=((2*ju) + 1), Bins=bins, Step=True, logScale=True)

 ## 3.CSE

Parameters, Hamiltonian, and the Unitaries

In [ ]:
# Parameters
ps = 2.5
js = 399.5
ks = 2.5
kps = 2
kpps = 3

# required operators
jz_s = qOps.Jz(js)
jy_s = qOps.Jy(js)
jx_s = qOps.Jx(js)
jzs_s = jz_s@jz_s

# Hamiltonians
h_s = (ps/js)*jzs_s
v_s = (ks/js)*(jzs_s + kps*(jx_s@jz_s + jz_s@jx_s) + kpps*(jx_s@jy_s + jy_s@jx_s))

# Unitaries
U_hs = liou.Liouvillian(h_s)
U_s = liou.Liouvillian(v_s)@U_hs

# Eigenvectors of the U_s
vecss = lina.eig(U_s.toarray())[1]

# components of eigenvectors in the Jz basis & taking the degeneracy into account
components = []
for i in range(len(vecss)):
    el = 0
    for k in range(int(len(vecss)/2)-2):
        p1 = (np.abs(vecss[:,i][el]))**2
        p2 = (np.abs(vecss[:,i][el+1]))**2
        el += 2
        components.append(p1+p2)

In [ ]:
# discrete PDF and log axes
sPlt.pltEig(components, Dims=((2*js) + 1), Bins=bins, Step=True, logScale=True)

# Eigenvector Statistics for Digital Ising Simulation 
---
    L. M. Sieberer, T. Olsacher, A. Elben, M. Heyl, P. Hauke, F. Haake, and P. Zoller
    npj Quantum Information volume 5, Article number: 78 (2019)
    https://www.nature.com/articles/s41534-019-0192-5
---
This digital quantum simulation maps to kicked-top, and the goal below is to find j value at which system is sensibly chaotic. <br/>

In [2]:

def IsingDS(jv, symp=False, chi=False):
    # parameters
    h_x = 0.1
    j_x = 0.7
    h_z = 0.3

    # Trotter step size
    tau = 5

    # required operators
    jz = qOps.Jz(jv)
    jx = qOps.Jx(jv)
    jzs = jz @ jz
    jxs = jx @ jx

    # Hamiltonians
    Ham_x = ( (h_x*jx) + ( (j_x*jxs)/( (2*jv) + 1) ) )
    Ham_z = ( (h_z*jz) + ( jzs/( (2*jv) + 1) ) ) 

    #Unitaries
    U_x = liou.Liouvillian(tau * Ham_x)
    U_z = liou.Liouvillian(tau * Ham_z)
    U_t = U_z @ U_x

    # Eigenvectors
    vecsi = lina.eig(U_t.toarray())[1]


    # components of eigenvectors in the Jz basis
    components = []
    if symp == True:
        if chi == False:
            el = 0
            for i in range(int(len(vecss)/2)):
                p1 = (np.abs(vecss[el].flatten()))**2
                p2 = (np.abs(vecss[el+1].flatten()))**2
                el += 2
                comps.append(sum(p1+p2))
        elif chi == True:
            for i in range(len(vecss)):
                el = 0
                for k in range(int(len(vecss)/2)-2):
                    p1 = (np.abs(vecss[:,i][el]))**2
                    p2 = (np.abs(vecss[:,i][el+1]))**2
                    el += 2
                    components.append(p1+p2)
    else:
        if chi == False:
            components = (np.abs(vecsi.flatten()))**2
        elif chi == True:
            for ind1 in range(len(vecsi)):
                components.append((np.abs(vecsi[ind1].flatten()))**2)

    return components

In [ ]:
def jInterval(jmin, jmax, logScale, xlim=1):
    print(xlim)
    fig = plt.figure(figsize=(12, 9))
    fig.text(0.5, 0.01, r'$|c_n|^2$', ha='center', fontsize=24)
    fig.text(0.005, 0.5, r'# of $|c_n|^2$', va='center', rotation='vertical', fontsize=24)

    axGrids = pltFncs.grid(2,2)
    jlist = np.arange(jmin,jmax,0.5)
    print(len(jlist))
    for ind in range(len(jlist)):
        components = IsingDS(jlist[ind])
        sPlt.pltEig(components, Dims=((2*jlist[ind]) + 1), Step=False, logScale=logScale, ax=axGrids[ind], Legend=False, xlim=xlim)
        print(ind)
    plt.show()

In [ ]:
jInterval(0.5, 2.5, False)

In [ ]:
jInterval(2.5, 4.5, False)

In [ ]:
jInterval(4.5, 6.5, False)

In [ ]:
jInterval(6.5, 8.5, False)

In [ ]:
jInterval(0.5, 2.5, True)

In [ ]:
jInterval(2.5, 4.5, True)

In [ ]:
jInterval(4.5, 6.5, True)

In [ ]:
jInterval(6.5, 8.5, True)